In [1]:
import os

In [2]:
%pwd

'/home/milad/projects/End-to-End-Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/milad/projects/End-to-End-Kidney-Disease-Classification'

In [44]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_pretrained: bool
    params_batch_size: int
    params_device: str

In [45]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_pretrained=self.params.PRETRAINED,
            params_batch_size=self.params.BATCH_SIZE,
            params_device=self.params.DEVICE
        )

        return prepare_base_model_config

In [47]:
import os
import urllib.request as request
from zipfile import ZipFile
from torchvision import models
import torch
import torch.nn as nn
from torchsummary import summary

In [54]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = models.vgg16(pretrained=self.config.params_pretrained)
        self.model = self.model.to(self.config.params_device)
        return self.model
    
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for param in model.parameters()[:-freeze_till]:
                param.requires_grad = False

        n_inputs = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.4),
        nn.Linear(256, classes), nn.LogSoftmax(dim=1))
        # multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
        return model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None
        )
        self.full_model = self.full_model.to(self.config.params_device)
        optimizer_top = torch.optim.Adam(self.full_model.parameters(), lr=self.config.params_learning_rate)
        checkpoint = {'model_state_dict': self.full_model.state_dict(),
                        'optimizer_state_dict': optimizer_top.state_dict()}
        summary(self.full_model, input_size=tuple(self.config.params_image_size), batch_size=self.config.params_batch_size, device=self.config.params_device)
        self.save_model(checkpoint=checkpoint, path=self.config.updated_base_model_path)

    
    @staticmethod
    def save_model(path: Path, checkpoint: dict):
        torch.save(checkpoint, path)

In [55]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-18 00:32:06,935: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-18 00:32:06,939: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-18 00:32:06,940: INFO: common: created directory at: artifacts]
[2024-02-18 00:32:06,942: INFO: common: created directory at: artifacts/prepare_base_model]


TypeError: _prepare_full_model() got an unexpected keyword argument 'learning_rate'